<a href="https://colab.research.google.com/github/howard-haowen/NLP-demos/blob/main/train_FAQ_model_with_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download a model

In [1]:
!pip install -q spacy

In [2]:
!python -m spacy download en_core_web_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-05 15:31:12.361275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 15:31:16.674066: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 15:31:16.674243: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

# Load a model

In [3]:
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
nlp = spacy.load("en_core_web_md")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

# Load a dataset

In [5]:
!wget https://download.microsoft.com/download/A/A/9/AA9C9802-B448-4361-89E0-DBF24D5A5C2C/WikiQACodePackage.zip
!unzip WikiQACodePackage.zip

--2023-03-05 15:31:47--  https://download.microsoft.com/download/A/A/9/AA9C9802-B448-4361-89E0-DBF24D5A5C2C/WikiQACodePackage.zip
Resolving download.microsoft.com (download.microsoft.com)... 69.192.209.23, 2600:1407:3c00:397::317f, 2600:1407:3c00:398::317f
Connecting to download.microsoft.com (download.microsoft.com)|69.192.209.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2072275 (2.0M) [application/octet-stream]
Saving to: ‘WikiQACodePackage.zip.1’

WikiQACodePackage.z 100%[===================>]   1.98M  --.-KB/s    in 0.08s   

2023-03-05 15:31:48 (24.1 MB/s) - ‘WikiQACodePackage.zip.1’ saved [2072275/2072275]

Archive:  WikiQACodePackage.zip
replace WikiQACodePackage/code/go_trec.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
import pandas as pd

In [7]:
def create_faq_format(input_path):
    faq_list = []
    
    with open(input_path) as f:
        lines = [line.strip('\n') for line in f]
    for i in range(2, len(lines)):
        l = lines[i].split('\t')
        if l[2]=="1":
            faq_list.append({"Question":l[0],"Answer":l[1]})
                
    return faq_list
    
qa_list = create_faq_format("WikiQACodePackage/data/wiki/WikiQASent-train.txt")
df = pd.DataFrame(qa_list, columns=["Question","Answer"])
df = df.drop_duplicates(subset='Question', keep="first")

In [11]:
df

,Question,Answer
0,how are glacier caves formed ?,A glacier cave is a cave formed within the ice...
1,how much is 1 tablespoon of water,This tablespoon has a capacity of about 15 mL .
4,how much are the harry potter movies worth,The series also originated much tie-in merchan...
5,how a rocket engine works,"A rocket engine , or simply `` rocket '' , is ..."
6,how are cholera and typhus transmitted and pre...,Transmission occurs primarily by drinking wate...
...,...,...
1032,what is melissa and joey about,The series follows local politician Mel Burke ...
1033,What is an economic feature ?,Other broad distinctions include those between...
1036,what is the average american income,"U.S. median household income fell from $ 51,14..."
1037,When was Apple Computer founded,"The company was founded on April 1 , 1976 , an..."


In [32]:
df.loc[872, 'Question']

'what is the disease osteonecrosis of the jaw ?'

# Convert the dataset to spaCy objects

In [20]:
from spacy.tokens import Doc

if not Doc.has_extension("answer"):
    Doc.set_extension("answer", default=None)

In [36]:
df.shape[0]

873

In [ ]:
questions = df['Question'].to_list()
questions

In [40]:
def convert_df2docs(df: pd.DataFrame, qcol: str, acol: str) -> list:
    """
    df: The dataset in DataFrame 
    qcol: Column name for questions
    acol: Column name for answers
    """
    questions = df[qcol].to_list()
    answers = df[acol].to_list()
    qa_tuples = [(q, {"answer": a}) for q, a in zip(questions, answers)]
    doc_tuples = nlp.pipe(qa_tuples, as_tuples=True,)
    docs = []
    for doc, context in doc_tuples:
        doc._.answer = context["answer"]
        docs.append(doc)
    return docs

In [41]:
docs = convert_df2docs(df, 'Question', 'Answer')

In [42]:
q1 = docs[1]
q1.text

'how much is 1 tablespoon of water'

In [43]:
q1._.answer

'This tablespoon has a capacity of about 15 mL .'

In [44]:
q1.vector

array([ 1.0898571e-01, -8.7722576e-01, -2.6414273e+00, -2.3487136e-01,
        4.9898944e+00, -1.5851715e+00,  8.0134708e-01,  3.9403498e+00,
       -3.3021432e-01, -9.1020584e-01,  7.9969711e+00,  1.8998331e+00,
       -1.0153342e+00,  3.6278853e-01, -1.7037715e-01, -2.1751431e-01,
        2.5533772e+00,  2.0110865e+00, -1.6006700e+00, -1.9568501e+00,
       -1.1215000e+00,  3.8781431e-01, -2.3038445e+00,  4.3978575e-01,
       -1.5860214e+00, -1.1137357e+00,  1.6348000e+00,  3.1899378e+00,
       -1.0150316e+00,  3.8759027e+00,  4.9552426e-01, -1.6056372e+00,
       -8.4532130e-01, -2.6308057e+00,  4.2058426e-01, -2.8168628e+00,
        2.5657177e-02,  3.8742044e+00,  2.5730498e+00,  2.6965156e+00,
       -1.9868528e+00,  6.4258152e-01, -8.8113278e-01, -1.5081428e-01,
       -2.6078515e+00,  5.4305573e+00,  3.8217087e+00, -7.2506338e-01,
        2.2679179e+00,  3.9423904e-01, -3.3076853e-01,  1.6842000e+00,
        1.4710844e+00, -2.9776883e+00, -9.2154139e-01, -3.6331374e-02,
      

# Create numpy 2D arrays for questions

In [45]:
import numpy as np

def create_2dvectors_from_docs(docs: list) -> np.array:
    doc_vectors1D = [doc.vector for doc in docs]
    doc_vectors2D = np.array(doc_vectors1D)
    return doc_vectors2D

In [46]:
doc_vectors2D = create_2dvectors_from_docs(docs)
doc_vectors2D.shape

(873, 300)

# Compute cosine similarity

In [47]:
from numpy.linalg import norm

def find_most_similar(query: str, docs: list, topK: int):
    query_vector = nlp(query).vector
    doc_vectors2D = create_2dvectors_from_docs(docs)
    cos_sim = np.dot(doc_vectors2D,query_vector)/(norm(doc_vectors2D, axis=1)*norm(query_vector))
    sorted_ids = np.argsort(cos_sim)
    top_sim_ids = sorted_ids[::-1][:topK] # in descending order
    print(f"Query: {query}")
    print("="*20)
    for id in top_sim_ids:
        score = cos_sim[id]
        doc = docs[id]
        print(f"Corpus question: {doc.text}")
        print(f"Corpus answer: {doc._.answer}")
        print(f"Score: {score}")
        print("="*10)

In [49]:
query = "What does my life insurance include?"
topK = 5
find_most_similar(query, docs, topK)

Query: What does my life insurance include?
Corpus question: what does life insurance cover ?
Corpus answer: Life insurance is a contract between an insured ( insurance policy holder ) and an insurer , where the insurer promises to pay a designated beneficiary a sum of money ( the `` benefits '' ) upon the death of the insured person .
Score: 0.8638259172439575
Corpus question: what does add my two cents mean
Corpus answer: `` My two cents '' ( 2¢ ) and its longer version `` put my two cents in '' is an United States ( US ) idiomatic expression , taken from the original English idiom expression : to put in `` my two pennies worth '' or `` my tuppence worth . ''
Score: 0.7863685488700867
Corpus question: what does salesforce.com do ?
Corpus answer: Though best known for its customer relationship management ( CRM ) product , Salesforce has also expanded into the `` social enterprise arena '' through acquisitions .
Score: 0.7341700792312622
Corpus question: what does Mazel tov ! mean
Corp